PROGRAMA_ANALISIS_LactAPP

Version: 05/12/2023

Test de funcionamiento Neo4j para analisis.

Necesario:

    - Docker Neo4j arrancado

    - Activado entorno venv

    - Licencia OpenAI (Archivo: .env)

Version 1.01

In [29]:
# Bibliotecas a Cargar

# Group 1: Web Framework (Flask) / Grupo 1: Framework Web (Flask)
from flask import Flask, request, jsonify, render_template
# Flask is a Python web framework used for building web applications.
# Imports include essential components for creating routes, handling HTTP requests, and rendering HTML templates.
# Flask es un framework web de Python que se utiliza para crear aplicaciones web.
# Las importaciones incluyen componentes esenciales para crear rutas, manejar solicitudes HTTP y renderizar plantillas HTML.

# Group 2: System Operations and Environment Variables / Grupo 2: Operaciones del Sistema y Variables de Entorno
import os
from dotenv import load_dotenv, find_dotenv
# This group handles system related operations and environment variables.
# 'os' provides functions for interacting with the operating system.
# 'dotenv' allows loading environment variables from '.env' files.
# Este grupo maneja operaciones relacionadas con el sistema y variables de entorno.
# 'os' proporciona funciones para interactuar con el sistema operativo.
# 'dotenv' permite cargar variables de entorno desde archivos '.env'.

# Group 3: Natural Language Processing (NLP) / Grupo 3: Procesamiento de Lenguaje Natural (NLP)
from langchain import (
    PromptTemplate, LLMChain, llms, text_splitter, vectorstores, chains, embeddings
)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# This group focuses on natural language processing (NLP) and utilizes the 'langchain' library.
# It includes various functionalities for text processing and language models.
# 'SentenceTransformer' is used for generating sentence vector representations for similarity calculations.
# 'cosine_similarity' is used to calculate cosine similarity between vectors.
# Este grupo se centra en el procesamiento de lenguaje natural (NLP) y utiliza la biblioteca 'langchain'.
# Incluye diversas funcionalidades para el procesamiento de texto y modelos de lenguaje.
# 'SentenceTransformer' permite generar representaciones vectoriales de oraciones para cálculos de similitud.
# 'cosine_similarity' se usa para calcular la similitud de coseno entre vectores.

# Group 4: Data Loading and Database Operations / Grupo 4: Carga de Datos y Operaciones de Base de Datos
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
# This group is responsible for loading documents and handling database operations.
# It includes functionalities for loading PDF documents ('PyPDFLoader') and working with databases like Neo4j.
# Este grupo se encarga de cargar documentos y manejar operaciones de base de datos.
# Incluye funcionalidades para cargar documentos PDF ('PyPDFLoader') y trabajar con bases de datos como Neo4j.

# Group 5: Additional Language Processing / Grupo 5: Procesamiento de Lenguaje Adicional
from langchain.llms import CTransformers
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
# This group includes additional imports related to 'langchain' and language models,
# like 'ChatOpenAI' for generating chat responses.
# Este grupo incluye importaciones adicionales relacionadas con 'langchain' y modelos de lenguaje,
# como 'ChatOpenAI' para generar respuestas de chat.


In [2]:
## Caso de usar .env file
## In case of using a .env file

# Carga de variables de Entorno 
# Load environment variables

load_dotenv(find_dotenv(), override=True)

# Accede a las variables de entorno en tu código
# Access environment variables in your code
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
URL_NEO4J = os.getenv('URL_NEO4J')
USERNAME_NEO4J = os.getenv('USERNAME_NEO4J')
PASSWORD_NEO4J = os.getenv('PASSWORD_NEO4J')

# Ahora, puedes usar las variables en tu script
# Now, you can use the variables in your script
#print(f'OPENAI_API_KEY: {OPENAI_API_KEY}')
#print(f'URL_NEO4J: {URL_NEO4J}')
#print(f'USERNAME_NEO4J: {USERNAME_NEO4J}')
#print(f'PASSWORD_NEO4J: {PASSWORD_NEO4J}')

1.- CARGA MODELO LOCAL

Modelo Local -> llm

In [3]:
# Initialize LLM and other components as in the original code
# Inicializa el LLM y otros componentes como en el código original

# Uncomment and set the appropriate model name for your local environment
# Descomenta y establece el nombre del modelo adecuado para tu entorno local

# local_llm = "neural-chat-7b-v3-1.Q4_K_M.gguf"
local_llm = "openhermes-2.5-mistral-7b.Q3_K_M.gguf"

In [4]:
# Model Parameters
# Parámetros del Modelo

# Define configuration parameters for the model
# Define los parámetros de configuración para el modelo
config = {
    'max_new_tokens': 512,       # Maximum number of new tokens generated
    'repetition_penalty': 1.1,   # Repetition penalty for text generation
    'temperature': 0,           # Temperature for controlling randomness (0 for deterministic output)
    'context_length': 1024,     # Length of the input context
    'stream': False             # Whether to stream the output or generate it all at once
}

| Parameter            | Type      | Description                                                     | Default   |
|----------------------|-----------|-----------------------------------------------------------------|-----------|
| top_k                | int       | The top-k value to use for sampling.                            | 40        |
| top_p                | float     | The top-p value to use for sampling.                            | 0.95      |
| temperature          | float     | The temperature to use for sampling.                            | 0.8       |
| repetition_penalty   | float     | The repetition penalty to use for sampling.                     | 1.1       |
| last_n_tokens        | int       | The number of last tokens to use for repetition penalty.        | 64        |
| seed                 | int       | The seed value to use for sampling tokens.                      | -1        |
| max_new_tokens       | int       | The maximum number of new tokens to generate.                   | 256       |
| stop                 | List[str] | A list of sequences to stop generation when encountered.        | None      |
| stream               | bool      | Whether to stream the generated text.                           | False     |
| reset                | bool      | Whether to reset the model state before generating text.        | True      |
| batch_size           | int       | The batch size to use for evaluating tokens in a single prompt. | 8         |
| threads              | int       | The number of threads to use for evaluating tokens.             | -1        |
| context_length       | int       | The maximum context length to use.                              | -1        |
| gpu_layers           | int       | The number of layers to run on GPU.                             | 0         |

In [5]:
# Initialize a CTransformers object with the specified parameters.
# Inicializa un objeto CTransformers con los parámetros especificados.
llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    lib="avx2",
    **config
)

# Print a message to indicate that LLM (Language Model) has been initialized.
# Imprime un mensaje para indicar que LLM (Modelo de Lenguaje) ha sido inicializado.
print("LLM Initialized....")

LLM Initialized....


2- MODELO EMBEDDINGS

In [8]:
# Define the model name for embeddings.
# Define el nombre del modelo para los embeddings.
model_name = "BAAI/bge-large-en"

# Specify keyword arguments for the model (e.g., device).
# Especifica argumentos clave para el modelo (por ejemplo, dispositivo).
model_kwargs = {'device': 'cpu'}

# Specify keyword arguments for encoding (e.g., normalization settings).
# Especifica argumentos clave para la codificación (por ejemplo, configuración de normalización).
encode_kwargs = {'normalize_embeddings': False}

# Initialize HuggingFaceBgeEmbeddings with the specified parameters.
# Inicializa HuggingFaceBgeEmbeddings con los parámetros especificados.
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

3.- DEFINICION PROMPTS

Para modelo local

Nota: El modelo local se utilizara en caso de que queramos generar una respuesta en modo texto usando el contexto de la cadana identificada como mas relevante para la pregunta indicada por la usuaria.

In [9]:
# Define a prompt template for generating prompts.
# Define una plantilla de prompt para generar preguntas.
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

# Specify the input variables used in the template.
# Especifica las variables de entrada utilizadas en la plantilla.
input_variables = ['context', 'question']

# Create a PromptTemplate object with the specified template and input variables.
# Crea un objeto PromptTemplate con la plantilla y las variables de entrada especificadas.
prompt = PromptTemplate(template=prompt_template, input_variables=input_variables)

4.- DATOS CONEXION Neo4j

La estructura de árboles de la aplicación LactApp se encuentra en una BBDD Neo4j

Hemos procedido a:

- Generar BBDD Neo4j en Docker

- Cargar sobre esa BBDD el DUMP proporcionado con los datos de la aplicación.

In [10]:
# Establish a connection to the Neo4j database.
# Establece una conexión a la base de datos Neo4j.
url = URL_NEO4J
username = USERNAME_NEO4J
password = PASSWORD_NEO4J

In [12]:
# Function to check the connection to the Neo4j database
# Función para verificar la conexión a la base de datos Neo4j
def verificar_conexion(url, username, password):
    try:
        # Attempt to establish a connection to the Neo4j database
        # Intento de establecer una conexión a la base de datos Neo4j
        with GraphDatabase.driver(url, auth=(username, password)) as driver:
            with driver.session() as session:
                # Run a test query to check the connection
                # Ejecutar una consulta de prueba para verificar la conexión
                session.run("RETURN 1")
        return True  # Return True if the connection is successful
    except Exception as e:
        # Handle any exceptions that occur during the connection attempt
        # Manejar cualquier excepción que ocurra durante el intento de conexión
        print(f"Connection error: {str(e)}")
        return False  # Return False if the connection fails

# Check the connection to the Neo4j database
# Verifica la conexión a la base de datos Neo4j
if verificar_conexion(url, username, password):
    print("Successful connection to the Neo4j database")
else:
    print("Failed to connect to the Neo4j database")

Successful connection to the Neo4j database


5.- INICIALIZACION EMBEDDING DE LA BBDD Neo4j

Procedemos a realizar el embedding de todos los nodos (Solo la primera vez)

Utilizaremos el modelo de embedding inicializado --> embeddings

Luego solo se realizara el embedding de los nodos que no dispongan de embedding ya calculado.

In [13]:
# Function to initialize the embedding for a node type
# Función para inicializar el embedding para un tipo de nodo
def init_node_embedding(node_label, text_properties):
    return Neo4jVector.from_existing_graph(
        embedding=embeddings,  # The embedding to use for initialization
        url=url,  # URL for the Neo4j database
        username=username,  # Username for database authentication
        password=password,  # Password for database authentication
        index_name=f"{node_label.lower()}_index",  # Name of the index for the node label
        node_label=node_label,  # Label of the node type
        text_node_properties=text_properties,  # Text properties for node embedding
        embedding_node_property="NEWembedding"  # Property name for the embedding in the database
    )

In [14]:
# Initialize embeddings for different types of nodes
# Inicializar embeddings para diferentes tipos de nodos

# Initialize embedding for "Choice" nodes with support for Spanish strings
Choice_embedding = init_node_embedding("Choice", ["string_es"])
print("Choice embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Choice" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

# Initialize embedding for "Question" nodes with support for Spanish strings
Question_embedding = init_node_embedding("Question", ["string_es"])
print("Question embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Question" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

# Initialize embedding for "Theme" nodes with support for Spanish strings
Theme_embedding = init_node_embedding("Theme", ["string_es"])
print("Theme embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Theme" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

Choice embedding OK
Question embedding OK
Theme embedding OK


6.- CALCULO SIMILITUD ENTRE EMBEDDING BBDD Neo4j y QUERY

Generaremos una propiedad 'similarity_query' en cada nodo.

El valor de esa propiedad sera la similitud entre el vector de la query y el EMBEDDING almacenado en cada nodo.

Se actualizara cada vez que se plantee una query nueva.

In [15]:
# Function to calculate similarity between the query and nodes
# Función para calcular la similitud entre la consulta y los nodos
def calculate_similarity(tx, input_query):
    # Get embeddings for the query
    # Obtener representaciones incrustadas (embeddings) para la consulta
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    # Obtain information for all nodes
    # Obtener información de todos los nodos
    result = tx.run("""
    MATCH (n)
    RETURN ID(n) AS node_id, n.NEWembedding AS node_embedding
    """)

    queries_to_update = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]

        # Calculate cosine similarity between the query and the node
        # Calcular la similitud del coseno entre la consulta y el nodo
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]

        # Prepare the query to update the node
        # Preparar la consulta para actualizar el nodo
        queries_to_update.append({
            "node_id": node_id,
            "score": similarity_score
        })

    # Update all nodes in a single transaction
    # Actualizar todos los nodos en una sola transacción
    for query_data in queries_to_update:
        tx.run("""
        MATCH (n)
        WHERE ID(n) = $node_id
        SET n.similarity_query = $score
        """, node_id=query_data["node_id"], score=query_data["score"])

7.- CALCULO NODOS MAS CERCANOS SEGUN MEDIA SIMILARIDAD INDICADA

- calculate_top_similarity_nodes --> Identifica los 5 nodos mas cercanos y lo devuelve en forma de lista 

In [16]:
# Function to calculate the list of closest nodes
# Función para calcular la lista de nodos más cercanos
def calculate_top_similarity_nodes(tx, input_query):
    # Get embeddings for the query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    result = tx.run("""
    MATCH (n)
    RETURN n.node_id AS node_id, n.NEWembedding AS node_embedding, n.similaridadquery AS similarity_score, n.string_es AS node_text
    ORDER BY similarity_score DESC
    LIMIT 5
    """)

    top_nodes = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]
        similarity_score = record["similarity_score"]
        textoNodo = record["node_text"]

        top_nodes.append({
            "node_id": node_id,
            "node_embedding": node_embedding,
            "similarity_score": similarity_score,
            "texto": textoNodo
        })

    return top_nodes

8.- CLASE PARA CALCULO CAMINOS OPTIMOS

Clase para identificar caminos optimos
    
    find_path -> Dado Nodo de inicio, profundidad maxima de busqueda devuelve lista de información de nodos para los caminos encontrados.
    
    get_all_themes_string -->  lista de cadenas de todos los Themas de la Neo4j
    
     get_node_ids_by_theme_string --> Devuelve una lista de IDs de nodos asociados con la cadena de tema proporcionada

In [17]:
class GraphPathFinder:
    def __init__(self, url, user, password):
        # Initialize the GraphPathFinder class with database connection parameters
        # Inicializar la clase GraphPathFinder con parámetros de conexión a la base de datos
        self.driver = GraphDatabase.driver(url, auth=(user, password))

    def close(self):
        # Close the database connection
        # Cerrar la conexión a la base de datos
        self.driver.close()

    def find_paths(self, start_node_id, max_depth, external_embedding):
        with self.driver.session() as session:
            query = """
            MATCH path = (n {node_id: $start_node_id})-[*1..%s]-(end)
            WHERE all(node in nodes(path) WHERE node.similaridadquery >= 0.7)
            RETURN [node in nodes(path) | [node.node_id, node.similaridadquery, node.string_es]] AS node_info
            ORDER BY reduce(s = 0, n IN nodes(path) | s + n.similaridadquery) DESC
            """ % max_depth
            result = session.run(query, start_node_id=start_node_id)
            # Return a list of node information for the found paths
            # Devolver una lista de información de nodos para los caminos encontrados
            return [record["node_info"] for record in result]

    def get_all_theme_strings(self):
        with self.driver.session() as session:
            query = """
            MATCH (theme:Theme)
            RETURN collect(theme.string_es) AS theme_strings
            """
            result = session.run(query)
            # Return a list of all theme strings in the database
            # Devolver una lista de todas las cadenas de temas en la base de datos
            return [record["theme_strings"] for record in result][0]

    def get_node_ids_by_theme_string(self, theme_string):
        with self.driver.session() as session:
            query = """
            MATCH (node:Theme {string_es: $theme_string})
            RETURN collect(node.node_id) AS node_ids
            """
            result = session.run(query, theme_string=theme_string)
            # Return a list of node IDs associated with the given theme string
            # Devolver una lista de IDs de nodos asociados con la cadena de tema proporcionada
            return [record["node_ids"] for record in result][0]

9.- PROCESADO NODOS

procesar_nodos --> Dado un nodo inicial, una profundidad máxima de nodo, y un número maximo de nodos a extraer nos devuelve la información.

In [18]:
def procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, external_embedding=None):
    """
    finder --> Clase de analisis caminos
    start_nodes_ids --> Nodo inicial donde empezar la busqueda
    Profund_Maxima_Nodos --> máximo nivel de profundidad de los nodos
    n_top_paths --> Numero de caminos a devolver
    external_embedding --> A utilizar en el futuro
    """

    resultados = []  # Almacenará los resultados finales

    # Itera sobre los nodos iniciales
    for start_node_id in start_node_ids:
        # Encuentra los caminos desde el nodo inicial
        paths = finder.find_paths(start_node_id, Profund_Maxima_Nodos, external_embedding)

        # Calcula la media de similaridad y almacena junto con el camino
        path_with_mean = []
        for path in paths:
            if path:
                # Calcula la media de la similaridad en el camino
                mean_similarity = sum(node[1] for node in path if node[1] is not None) / len(path)
                path_with_mean.append((path, mean_similarity))

        # Ordena los caminos por la media de similaridad de mayor a menor
        path_with_mean.sort(key=lambda x: x[1], reverse=True)

        # Guarda la información de los n caminos con mayor media
        top_paths = path_with_mean[:n_top_paths]
        for path, mean in top_paths:
            path_info = [(node[0], node[1]) for node in path]
            string_es_values = [node[2] for node in path if node[2] is not None]
            max_similarity_node = max(path, key=lambda x: x[1])
            max_similarity_node_info = (max_similarity_node[0], max_similarity_node[1], max_similarity_node[2])

            resultados.append({
                'start_node_id': start_node_id,
                'path_info': path_info,
                'mean_similarity': mean,
                'string_es_values': string_es_values,
                'max_similarity_node_info': max_similarity_node_info
            })

    return resultados  # Devuelve los resultados finales

In [19]:
# Visualización de resultados formato texto.

def mostrar_resultado(resultado, mostrar):
    ''' Imprime el resultado de la búsqueda de caminos en la base de datos Neo4j
    limitando el número de nodos del camino mostrados
    y valores de string_es a mostrar.
    el valor limite dado por: 'mostrar'
    '''
    print("ID del Nodo Inicial:", resultado['start_node_id'])

    print("\nInformación del Camino (mostrando hasta {} nodos):".format(mostrar))
    for node in resultado['path_info'][:mostrar]:
        print(f"  Nodo ID: {node[0]}, Similaridad: {node[1]}")

    print("\nMedia de Similaridad:", resultado['mean_similarity'])

    print("\nValores de string_es (mostrando hasta {} valores):".format(mostrar))
    for value in resultado['string_es_values'][:mostrar]:
        print(f"  {value}")

    print("\nInformación del Nodo con Máxima Similaridad:")
    max_node_info = resultado['max_similarity_node_info']
    print(f"  Nodo ID: {max_node_info[0]}, Similaridad: {max_node_info[1]}")
    print(f"  string_es: {max_node_info[2]}")

A.- PROCESO COMPLETO DE ANALISIS EN SECUENCIA

A1.- Identificación Mensajes Usuarias disponibles

In [20]:
FILEDATOS = '/media/ubuntu/Linux3/02_TFM/TEST_BIBLIOTECA/MODELOS/a.xlsx'

import pandas as pd
import numpy as np
df = pd.read_excel(FILEDATOS)

In [22]:
mensajes = df['message'].to_list()
mensajes = list(set(mensajes))

A2.- GENERACION DE CLASIFICACION MENSAJE USUARIA USANDO LLMs

In [24]:
# Carga Modelo Open AI

#modelOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

modelOpenAI = ChatOpenAI(temperature=0)

In [30]:
# Ejemplo de Prompts condicionales

#Ref: https://python.langchain.com/docs/guides/local_llms
# Mistral --> https://smith.langchain.com/hub/rlm/rag-prompt-mistral
# Llama --> https://smith.langchain.com/hub/rlm/rag-prompt-llama


DEFAULT_MISTRAL_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template="""<s> [INST] You are an expert in breastfeeding.\
If you don't know the answer, just say that you don't know. \
The output must be only the category. [/INST] </s> \
[INST] Question: classify the text: {texto}. \
Context: The categories for classify are: {categorias}. \
Answer: [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template=""""Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: """,
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[
        (lambda model: (model == llm), DEFAULT_MISTRAL_SEARCH_PROMPT)
    ],
)


prompt2 = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt3 = QUESTION_PROMPT_SELECTOR.get_prompt(modelOpenAI)



print('Mistral7:\n', prompt2)

print('OpenAI:\n', prompt3)

Mistral7:
 input_variables=['categorias', 'texto'] template="<s> [INST] You are an expert in breastfeeding.If you don't know the answer, just say that you don't know. The output must be only the category. [/INST] </s> [INST] Question: classify the text: {texto}. Context: The categories for classify are: {categorias}. Answer: [/INST]"
OpenAI:
 input_variables=['categorias', 'texto'] template='"Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: '


In [31]:
# Caso Mistral y Llama2

chainMistral = prompt2 | llm | StrOutputParser()

chainOpenAI = prompt3 | modelOpenAI | StrOutputParser()

In [32]:
def clasificar_LLMs(texto, categorias, chain):
    '''Función de clasificación
    Entrada:
        texto -> Conversación Usuaria
        categorias --> Lista de categorias donde se ha de clasificar
        chain --> Cadena de LangChain
    respuesta:
        Respuesta del modelo
    '''
    response = chain.invoke({"categorias": categorias, "texto": texto})
    return response

In [33]:
# Mensaje de TEST

entrada = mensajes[0]

CREACION LISTA CON TODAS LAS CATEGORIAS EN BBDD Neo4j

In [34]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
categorias = finder.get_all_theme_strings()
categorias = list(set(categorias))
print(categorias)
print(len(categorias))

['EMBARAZO: SEGUIR LACTANDO', 'CONSERVACIÓN', 'Semana Mundial de la Lactancia', 'MIS SENTIMIENTOS', 'Coronavirus', 'GEMELOS, MELLIZOS', 'Mi embarazo por etapas', 'LOS PRIMEROS DÍAS', 'PORTEO', 'Planifica tu vuelta al trabajo', 'MI BEBÉ MAMA MUCHO', '¿TENGO POCA LECHE?', 'Punto Violeta', 'LA VUELTA AL TRABAJO', 'GRIETAS Y HERIDAS', 'SUEÑO Y LACTANCIA', 'COSAS QUE HACE MI BEBÉ', 'BACHES Y CRISIS', 'PECHO: FORMA Y TAMAÑO', 'A PARTIR DE 6 MESES', 'Guía de regalos', 'EXTRACCIÓN DE LECHE', 'DEJAR LA FÓRMULA', 'Lactancia artificial', 'MI BEBÉ ES PREMATURO', 'INDUCCIÓN', 'ME HE QUEDADO EMBARAZADA', 'EL DESTETE POR ETAPAS', 'DEJAR DE DAR EL PECHO', 'PRUEBAS DIAGNÓSTICAS', 'CÓMO PRODUCIR MÁS LECHE', 'SEGUIR PREPARÁNDOME', 'MI BEBÉ NO GANA PESO', 'LA DONACIÓN DE LECHE', '¡QUIERO SER DONANTE!', 'LACTANCIA MIXTA', 'EN QUÉ DARLE LA LECHE', 'Cómo gestionar mi mastitis', 'Baby Led Weaning', 'SALUD DE MI BEBÉ', 'GUÍA DEL DOLOR AL AMAMANTAR', 'SOBRE MI SALUD', 'DOLOR EN EL PECHO', '

IDENTIFICACION DE CLASIFICACION DEL MENSAJE DE LA USUARIA

entrada --> Mensaje de entrada

categorias --> Lista de todas las categorias identificadas en Neo4j

chainOpenAI --> Cadena de analisis con OpenAI

In [35]:
salidaOpen= clasificar_LLMs(entrada, categorias, chainOpenAI)

IDENTIFICACION NODO CON MAYOR SIMILITUD

In [36]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
ids = finder.get_node_ids_by_theme_string(salidaOpen)
ids = list(set(ids))
print(ids)
print(len(ids))

['t7']
1


Procesado para varios mensajes (Modo TEST)

In [37]:
for mensaje in mensajes[:1]:
    print("MENSAJE:", mensaje)
    print("-------------------------")
    # Query que deseas comparar con las propiedades de los nodos
    query = mensaje
    # Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
    with GraphDatabase.driver(url, auth=(username, password)) as driver:
        embeddings = SentenceTransformer(model_name, **model_kwargs)

        with driver.session() as session:
            session.execute_write(calculate_similarity, query)

    print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

    # Identificación de los temas de la pregunta
    salidaOpen= clasificar_LLMs(mensaje, categorias, chainOpenAI)

    # Creación de la instancia de GraphPathFinder
    finder = GraphPathFinder(url, username, password)
    ids = finder.get_node_ids_by_theme_string(salidaOpen)

    # Lista de identificadores de nodo iniciales
    start_node_ids = ids  # Lista de nodos iniciales

    # Profundidad máxima de búsqueda
    Profund_Maxima_Nodos = 7

    # Número de caminos con mayor media de similaridad a mostrar
    n_top_paths = 5

    # Llamada a la función procesar_nodos
    resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths)

    mostrar_resultado(resultados[0], Profund_Maxima_Nodos)
    # Imprimir resultados
    #for resultado in resultados:
    #    print(resultado)

    # Cerrar la conexión
    finder.close()

MENSAJE: Hola, buenos días.
Mi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al terminar durante un rato, quizás 15-20 min. No tengo grietas, se trata de un dolor punzante. He probado varias posiciones y, cuando abre mucho la boca para engancharse no va del todo mal, por lo menos el dolor no es intenso. ¿Cómo puedo hacer para conseguir la mejor apertura de su boca y evitar dolor? Gracias 
-------------------------
Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Hola, buenos días.
Mi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al te

RESUMEN RESULTADOS

In [38]:
# PREGUNTAS DE PRUEBA
print("Pregunta:", query, "\n")

Pregunta: Hola, buenos días.
Mi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al terminar durante un rato, quizás 15-20 min. No tengo grietas, se trata de un dolor punzante. He probado varias posiciones y, cuando abre mucho la boca para engancharse no va del todo mal, por lo menos el dolor no es intenso. ¿Cómo puedo hacer para conseguir la mejor apertura de su boca y evitar dolor? Gracias  



In [39]:
# Tematica identificada
print('Tematica:', salidaOpen)

Tematica: DOLOR EN EL PECHO


In [40]:
# Nodo de tematica identificada
print('Nodo:', ids)

Nodo: ['t7']


In [41]:
# Nodos con mayor similaridad
for t in range(0, n_top_paths):
    tuplas = resultados[t]['path_info']
    # Extracción del primer elemento de cada tupla
    primeros_elementos= [tupla[0] for tupla in tuplas]
    # Mostrar la nueva lista
    print('Nodos camino identificado:', primeros_elementos)

Nodos camino identificado: ['t7', 'q122', 'c497', 'r628']
Nodos camino identificado: ['t7', 'q122', 'c498', 'r630']
Nodos camino identificado: ['t7', 'q122', 'c496', 'r627']
Nodos camino identificado: ['t7', 'q122', 'c495', 'r626']
Nodos camino identificado: ['t7', 'q122']


In [42]:
# CADENA TEXTOS
print('Numero de cadenas:', n_top_paths)
# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    print('Cadena: ',n, "Similaridad Media: ",resultados[n]['mean_similarity'], '\n')
    print("Pregunta: ", query)
    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        # Extracción del primer elemento de cada tupla
        primeros_elementos= [tupla[0] for tupla in tuplas]
        print(t,': (Nodo:', primeros_elementos[t],')', texto)
    print('-------------------------')


Numero de cadenas: 5
Cadena:  0 Similaridad Media:  0.8742373587220464 

Pregunta:  Hola, buenos días.
Mi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al terminar durante un rato, quizás 15-20 min. No tengo grietas, se trata de un dolor punzante. He probado varias posiciones y, cuando abre mucho la boca para engancharse no va del todo mal, por lo menos el dolor no es intenso. ¿Cómo puedo hacer para conseguir la mejor apertura de su boca y evitar dolor? Gracias 
0 : (Nodo: t7 ) DOLOR EN EL PECHO
1 : (Nodo: q122 ) Sobre la corriente que sientes en el pecho:
2 : (Nodo: c497 ) Lo siento durante y entre las tomas. No es muy doloroso
3 : (Nodo: r628 ) 
 Por lo que has indicado en LactApp, creemos que se trata de la sensación que produce el reflejo de salida o eyección de leche . 

In [43]:
# Extracción del segundo elemento de cada tupla
segundos_elementos= [tupla[1] for tupla in tuplas]
print('Similaridad:', segundos_elementos) 

Similaridad: [0.8527326575391612, 0.881504183846252]


In [44]:
# Similaridad media
print('Similaridad media:', resultados[0]['mean_similarity'])

Similaridad media: 0.8742373587220464


In [45]:
# Valores de string_es
print('Texto Nodos:', resultados[0]['string_es_values'])

Texto Nodos: ['DOLOR EN EL PECHO', 'Sobre la corriente que sientes en el pecho:', 'Lo siento durante y entre las tomas. No es muy doloroso', '\n Por lo que has indicado en LactApp, creemos que se trata de la sensación que produce el reflejo de salida o eyección de leche . Es una sensación más aguda las primeras semanas de lactancia y, a medida que pasan los meses, va disminuyendo. \n Esta sensación eléctrica, que puede llegar a ser bastante dolorosa, se puede repetir a lo largo de la toma, cuando se producen nuevos reflejos de eyección de la leche. \n También la puedes sentir momentos antes de que el bebé se despierte para pedir el pecho, o si oyes llorar a un bebé, aunque no sea el tuyo. El cuerpo de la madre se sincroniza con el del bebé. \n Si el dolor aumenta o se intensifica, no dudes en consultar de nuevo LactApp, acudir a un grupo de apoyo a la lactancia o consultar con una IBCLC o comadrona.']


SALIDAS EN FORMATO JSON

El objetivo es poder interactuar en la aplicacion WEB

In [46]:
import json

# CADENA TEXTOS
n_top_paths = 3  # Cambia esto según tu necesidad

# Crear una lista para almacenar los datos de salida
output_data = []

# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    cadena_data = {
        'Cadena': n,
        'Similaridad_Media': resultados[n]['mean_similarity'],
        'Pregunta': query,
        'Nodos_Textos': []
    }

    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        primeros_elementos = [tupla[0] for tupla in tuplas]
        nodo_texto = {
            'Nodo': primeros_elementos[t],
            'Texto': texto
        }
        cadena_data['Nodos_Textos'].append(nodo_texto)

    output_data.append(cadena_data)

# Serializar los datos de salida como JSON
json_output = json.dumps(output_data, ensure_ascii=False, indent=4)

# Imprimir la salida JSON
print(json_output)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8742373587220464,
        "Pregunta": "Hola, buenos días.\nMi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al terminar durante un rato, quizás 15-20 min. No tengo grietas, se trata de un dolor punzante. He probado varias posiciones y, cuando abre mucho la boca para engancharse no va del todo mal, por lo menos el dolor no es intenso. ¿Cómo puedo hacer para conseguir la mejor apertura de su boca y evitar dolor? Gracias ",
        "Nodos_Textos": [
            {
                "Nodo": "t7",
                "Texto": "DOLOR EN EL PECHO"
            },
            {
                "Nodo": "q122",
                "Texto": "Sobre la corriente que sientes en el pecho:"
            },
            {
                "Nodo": "c

In [47]:
# Cargar el JSON
data = json.loads(json_output)

# Imprimir el JSON formateado en pantalla
formatted_json = json.dumps(data, ensure_ascii=False, indent=4)
print(formatted_json)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8742373587220464,
        "Pregunta": "Hola, buenos días.\nMi bebé tiene 23 días, hace una semana la pediatra detectó un frenillo lingual tipo 1. La lactancia era indolora hasta hace 4 días. Con el pecho izquierdo vamos bien, no hay dolor. Pero con el derecho, el pezón me duele al engancharse el bebé, durante la toma y al terminar durante un rato, quizás 15-20 min. No tengo grietas, se trata de un dolor punzante. He probado varias posiciones y, cuando abre mucho la boca para engancharse no va del todo mal, por lo menos el dolor no es intenso. ¿Cómo puedo hacer para conseguir la mejor apertura de su boca y evitar dolor? Gracias ",
        "Nodos_Textos": [
            {
                "Nodo": "t7",
                "Texto": "DOLOR EN EL PECHO"
            },
            {
                "Nodo": "q122",
                "Texto": "Sobre la corriente que sientes en el pecho:"
            },
            {
                "Nodo": "c